In [1]:
import sys
sys.path.append("..")

In [2]:
from collections import defaultdict
from itertools import islice

from resources.utils import get_puzzle_input

### Part 1


The year 518 is significantly more underground than your history books implied. Either that, or you've arrived in a vast cavern network under the North Pole.

After exploring a little, you discover a long tunnel that contains a row of small pots as far as you can see to your left and right. A few of them contain plants - someone is trying to grow things in these geothermally-heated caves.

The pots are numbered, with 0 in front of you. To the left, the pots are numbered -1, -2, -3, and so on; to the right, 1, 2, 3.... Your puzzle input contains a list of pots from 0 to the right and whether they do (#) or do not (.) currently contain a plant, the initial state. (No other pots currently contain plants.) For example, an initial state of #..##.... indicates that pots 0, 3, and 4 currently contain plants.

Your puzzle input also contains some notes you find on a nearby table: someone has been trying to figure out how these plants spread to nearby pots. Based on the notes, for each generation of plants, a given pot has or does not have a plant based on whether that pot (and the two pots on either side of it) had a plant in the last generation. These are written as LLCRR => N, where L are pots to the left, C is the current pot being considered, R are the pots to the right, and N is whether the current pot will have a plant in the next generation. For example:

A note like ..#.. => . means that a pot that contains a plant but with no plants within two pots of it will not have a plant in it during the next generation.
A note like ##.## => . means that an empty pot with two plants on each side of it will remain empty in the next generation.
A note like .##.# => # means that a pot has a plant in a given generation if, in the previous generation, there were plants in that pot, the one immediately to the left, and the one two pots to the right, but not in the ones immediately to the right and two to the left.
It's not clear what these plants are for, but you're sure it's important, so you'd like to make sure the current configuration of plants is sustainable by determining what will happen after 20 generations.

For example, given the following input:

```
initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #
For brevity, in this example, only the combinations which do produce a plant are listed. (Your input includes all possible combinations.) Then, the next 20 generations will look like this:

                 1         2         3     
       0         0         0         0     
 0: ...#..#.#..##......###...###...........
 1: ...#...#....#.....#..#..#..#...........
 2: ...##..##...##....#..#..#..##..........
 3: ..#.#...#..#.#....#..#..#...#..........
 4: ...#.#..#...#.#...#..#..##..##.........
 5: ....#...##...#.#..#..#...#...#.........
 6: ....##.#.#....#...#..##..##..##........
 7: ...#..###.#...##..#...#...#...#........
 8: ...#....##.#.#.#..##..##..##..##.......
 9: ...##..#..#####....#...#...#...#.......
10: ..#.#..#...#.##....##..##..##..##......
11: ...#...##...#.#...#.#...#...#...#......
12: ...##.#.#....#.#...#.#..##..##..##.....
13: ..#..###.#....#.#...#....#...#...#.....
14: ..#....##.#....#.#..##...##..##..##....
15: ..##..#..#.#....#....#..#.#...#...#....
16: .#.#..#...#.#...##...#...#.#..##..##...
17: ..#...##...#.#.#.#...##...#....#...#...
18: ..##.#.#....#####.#.#.#...##...##..##..
19: .#..###.#..#.#.#######.#.#.#..#.#...#..
20: .#....##....#####...#######....#.#..##.
```

The generation is shown along the left, where 0 is the initial state. The pot numbers are shown along the top, where 0 labels the center pot, negative-numbered pots extend to the left, and positive pots extend toward the right. Remember, the initial state begins at pot 0, which is not the leftmost pot used in this example.

After one generation, only seven plants remain. The one in pot 0 matched the rule looking for ..#.., the one in pot 4 matched the rule looking for .#.#., pot 9 matched .##.., and so on.

In this example, after 20 generations, the pots shown as # contain plants, the furthest left of which is pot -2, and the furthest right of which is pot 34. Adding up all the numbers of plant-containing pots after the 20th generation produces 325.

After 20 generations, what is the sum of the numbers of all pots which contain a plant?

In [3]:
test_input = """initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #"""

In [4]:
def parse_lines(lines):
    initial_state = lines[0][15:]
    instructions = [tuple(line.split(' => ')) for line in lines[2:]]
    # Just keep the positive ones
    instructions = [neighbours for (neighbours, result) in instructions if result == '#']
    return initial_state, instructions

In [5]:
test_lines = test_input.split('\n')
test_state, test_instructions = parse_lines(test_lines)
test_state, test_instructions

('#..#.#..##......###...###',
 ['...##',
  '..#..',
  '.#...',
  '.#.#.',
  '.#.##',
  '.##..',
  '.####',
  '#.#.#',
  '#.###',
  '##.#.',
  '##.##',
  '###..',
  '###.#',
  '####.'])

In [6]:
puzzle_input = get_puzzle_input('/tmp/day_12.txt')
puzzle_state, puzzle_instructions = parse_lines(puzzle_input)
puzzle_state, puzzle_instructions

('##.#....#..#......#..######..#.####.....#......##.##.##...#..#....#.#.##..##.##.#.#..#.#....#.#..#.#',
 ['#####',
  '.#..#',
  '#..#.',
  '#...#',
  '...##',
  '##..#',
  '.#.##',
  '#.###',
  '.##.#',
  '.#...',
  '##...',
  '##.##',
  '##.#.',
  '#.##.'])

In [7]:
class PlantPots:
    def __init__(self, initial_state, instructions):
        self.min_plant = 0 
        self.max_plant = len(initial_state)
        self.instructions = set(instructions)
        
        self.state = defaultdict(lambda *_: '.')
        for x in range(len(initial_state)):
            self.state[x] = initial_state[x]

    def is_plant(self, idx):
        return self.state[idx] == '#'
    
    def will_be_plant(self, idx):
        neighbours = []
        for x in range(idx - 2, idx + 3):
            neighbours.append(self.state[x])
            
        return ''.join(neighbours) in self.instructions

    @property
    def checksum(self):
        cs = 0
        for x in range(self.min_plant, self.max_plant + 1):
            if self.is_plant(x):
                cs += x
        
        return cs
    
    def __repr__(self):
        output = []
        for x in range(self.min_plant, self.max_plant + 1):
            output.append(self.state[x])
        return ''.join(output)

    def __iter__(self):
        return self
    
    def __next__(self):
        next_state = defaultdict(lambda *_: '.')
        for x in range(self.min_plant - 2, self.max_plant + 3):
            is_plant = self.will_be_plant(x)
            if x < self.min_plant and is_plant:
                self.min_plant = x
            if x > self.max_plant and is_plant:
                self.max_plant = x
            
            if is_plant:
                next_state[x] = '#'
                
        self.state = next_state
        
        return self


In [8]:
test_pots = PlantPots(test_state, test_instructions)
test_pots

#..#.#..##......###...###.

In [9]:
assert test_pots.is_plant(3)
assert not test_pots.will_be_plant(3)

In [10]:
next(test_pots)
assert str(test_pots) == '#...#....#.....#..#..#..#.'

In [11]:
test_pots = PlantPots(test_state, test_instructions)
list(islice(test_pots, 20, 20))
test_pots.checksum

325

In [12]:
puzzle_pots = PlantPots(puzzle_state, puzzle_instructions)
list(islice(puzzle_pots, 20, 20))
puzzle_pots.checksum

3241

### Part 2

In [13]:
puzzle_pots = PlantPots(puzzle_state, puzzle_instructions)
seen_states = set()
states = []

x = 1
for _ in islice(puzzle_pots, 200):
    state = str(puzzle_pots), puzzle_pots.checksum
    states.append(state)
    
    if x > 1:
        print(x, state[1], state[1] - states[-2][1])
    
    x+= 1


2 2169 91
3 2230 61
4 2655 425
5 2896 241
6 2986 90
7 2932 -54
8 2708 -224
9 2677 -31
10 2766 89
11 2197 -569
12 2581 384
13 2687 106
14 2813 126
15 3188 375
16 3212 24
17 2909 -303
18 2857 -52
19 3131 274
20 3241 110
21 3045 -196
22 3200 155
23 2844 -356
24 3197 353
25 2967 -230
26 3394 427
27 3485 91
28 2935 -550
29 2558 -377
30 2696 138
31 3274 578
32 3214 -60
33 2920 -294
34 3096 176
35 3172 76
36 3032 -140
37 2919 -113
38 2661 -258
39 2596 -65
40 2458 -138
41 2812 354
42 2987 175
43 3196 209
44 3296 100
45 3243 -53
46 3130 -113
47 3449 319
48 3219 -230
49 3291 72
50 3431 140
51 3438 7
52 3152 -286
53 3258 106
54 3359 101
55 3662 303
56 3587 -75
57 3661 74
58 3660 -1
59 3628 -32
60 3857 229
61 3947 90
62 3941 -6
63 3855 -86
64 3925 70
65 4187 262
66 4276 89
67 4310 34
68 4275 -35
69 4366 91
70 4338 -28
71 4286 -52
72 4313 27
73 4298 -15
74 4300 2
75 4457 157
76 4569 112
77 4558 -11
78 4384 -174
79 4366 -18
80 4485 119
81 4725 240
82 4788 63
83 4717 -71
84 4557 -160
85 4692 135
86 4

In [14]:
# From the 117th generation the checksum increments by 55 each time
# it's a 'floater' moving to the right each time
(50000000000 - 117) * 55 + 6346

2749999999911